In [ ]:
# math
import math
import pandas as pd
import datetime

from experiments_2022 import IMAGE_PATH
from experiments_2022.zone_level_analysis import (
    cleaning,
    viz,
)
from experiments_2022.datasets import (
    load_weather,
    pull_from_dataset,
)

from utils import (
    PROJECTS_2022,
    PROJECTS_2022_TOTAL,
    gini,
    add_total_to_dfs,
    SINGLE_PLOT_LEGEND_SIZE,
    SINGLE_PLOT_TXT_SIZE,
    MULTI_PLOT_TITLE_SIZE,
    MULTI_PLOT_LEGEND_SIZE,
    MULTI_PLOT_TXT_SIZE,
)

# Weather

In [ ]:
weather = load_weather("2023")
weather = weather.loc[: pd.Timestamp("2023-12-31 23:00:00"), :]
weather = cleaning.clean_df(weather, only_business_hours=True, no_weekends=True)
weather_daily = weather.resample("D").mean()
temps_daily = weather_daily["temperature"]
RH_daily = weather_daily["RH"]

In [ ]:
percentiles = [0, 0.25, 0.5, 0.75, 1]
temps = {}
humids = {}
for p in percentiles:
    temps[p] = float(temps_daily.quantile(p))
    humids[p] = float(RH_daily.quantile(p))

In [ ]:
days_temp = {}
for p in range(len(percentiles) - 1):
    this_p = percentiles[p]
    next_p = percentiles[p + 1]
    mask = (temps_daily >= temps[this_p]) & (temps_daily < temps[next_p])
    filtered = temps_daily[mask]
    days_temp[this_p] = list(filtered.index)

hours_temp = {}
for bucket in days_temp:
    these_days = days_temp[bucket]
    day_dates = set(pd.to_datetime(these_days).date)
    mask = pd.Index(weather.index.date).isin(day_dates)
    hours_temp[bucket] = weather.index[mask]

In [ ]:
days_RH = {}
for p in range(len(percentiles) - 1):
    this_p = percentiles[p]
    next_p = percentiles[p + 1]
    mask = (RH_daily >= humids[this_p]) & (RH_daily < humids[next_p])
    filtered = RH_daily[mask]
    days_RH[this_p] = list(filtered.index)

hours_RH = {}
for bucket in days_RH:
    these_days = days_RH[bucket]
    day_dates = set(pd.to_datetime(these_days).date)
    mask = pd.Index(weather.index.date).isin(day_dates)
    hours_RH[bucket] = weather.index[mask]

In [ ]:
def F_to_C(F):
    return (F - 32) * 5 / 9

# Cooling requests

In [ ]:
CRs = pull_from_dataset(
    "2023",
    PROJECTS_2022,
    "zone-simple_cooling_requests",
)

In [ ]:
CRs = add_total_to_dfs(CRs)

## OAT

In [ ]:
CRs_bucketed_temp = {}
CRs_lorenz_bucketed_temp = {}
for project in CRs:
    CRs_bucketed_temp[project] = {}
    CRs_lorenz_bucketed_temp[project] = {}
    for bucket in hours_temp:
        these_CRs = CRs[project].loc[hours_temp[bucket], :].sum()
        CRs_bucketed_temp[project][bucket] = these_CRs.to_frame(name=bucket)
        these_CRs = these_CRs.sort_values(ascending=False)
        lorenz = these_CRs.cumsum() / these_CRs.sum()
        CRs_lorenz_bucketed_temp[project][bucket] = lorenz.to_frame(name=bucket)

In [ ]:
CRs_bucketed_df_temp = {}
CRs_lorenz_bucketed_df_temp = {}
for project in CRs_bucketed_temp:
    these_CRs = CRs_bucketed_temp[project]
    df = pd.concat([these_CRs[bucket] for bucket in these_CRs], axis=1)
    CRs_bucketed_df_temp[project] = df

    these_lorenz = CRs_lorenz_bucketed_temp[project]
    df = pd.concat([these_lorenz[bucket] for bucket in these_lorenz], axis=1)
    CRs_lorenz_bucketed_df_temp[project] = df

In [ ]:
fig = viz.make_dot_plot(
    y_data=CRs_lorenz_bucketed_df_temp,
    sort_by="all",
    normalize_x=True,
    y_axis_title="Fraction of Cumulative<br>Cooling Requests [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.075,
    horizontal_spacing=0.1,
    color_legend={
        "name": {
            0.0: f"Q1 OAT={round(F_to_C(temps[0]))}°C-{round(F_to_C(temps[0.25]))}°C",
            0.25: f"Q2 OAT={round(F_to_C(temps[0.25]))}°C-{round(F_to_C(temps[0.5]))}°C",
            0.5: f"Q3 OAT={round(F_to_C(temps[0.5]))}°C-{round(F_to_C(temps[0.75]))}°C",
            0.75: f"Q4 OAT={round(F_to_C(temps[0.75]))}°C-{round(F_to_C(temps[1]))}°C",
        },
        "color": {
            0.0: "#1f77b4",  # muted blue
            0.25: "#ff7f0e",  # orange
            0.5: "#d62728",  # red
            0.75: "#6a3d9a",  # purple
        },
    },
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureF2.png")

In [ ]:
# fig

In [ ]:
CRs_ginis_temp = pd.DataFrame(index=PROJECTS_2022_TOTAL, columns=[0, 0.25, 0.5, 0.75])
for project in list(CRs_ginis_temp.index):
    for bucket in list(CRs_ginis_temp.columns):
        CRs_ginis_temp.loc[project, bucket] = gini(
            CRs_bucketed_df_temp[project][bucket]
        )

## RH

In [ ]:
CRs_bucketed_RH = {}
CRs_lorenz_bucketed_RH = {}
for project in CRs:
    CRs_bucketed_RH[project] = {}
    CRs_lorenz_bucketed_RH[project] = {}
    for bucket in hours_temp:
        these_CRs = CRs[project].loc[hours_RH[bucket], :].sum()
        CRs_bucketed_RH[project][bucket] = these_CRs.to_frame(name=bucket)
        these_CRs = these_CRs.sort_values(ascending=False)
        lorenz = these_CRs.cumsum() / these_CRs.sum()
        CRs_lorenz_bucketed_RH[project][bucket] = lorenz.to_frame(name=bucket)

In [ ]:
CRs_bucketed_df_RH = {}
CRs_lorenz_bucketed_df_RH = {}
for project in CRs_bucketed_RH:
    these_CRs = CRs_bucketed_RH[project]
    df = pd.concat([these_CRs[bucket] for bucket in these_CRs], axis=1)
    CRs_bucketed_df_RH[project] = df

    these_lorenz = CRs_lorenz_bucketed_RH[project]
    df = pd.concat([these_lorenz[bucket] for bucket in these_lorenz], axis=1)
    CRs_lorenz_bucketed_df_RH[project] = df

In [ ]:
fig = viz.make_dot_plot(
    y_data=CRs_lorenz_bucketed_df_RH,
    sort_by="all",
    normalize_x=True,
    y_axis_title="Fraction of Cumulative<br>Cooling Requests [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.075,
    horizontal_spacing=0.1,
    color_legend={
        "name": {
            0.0: f"Q1 RH={round(humids[0])}%-{round(humids[0.25])}%",
            0.25: f"Q2 RH={round(humids[0.25])}%-{round(humids[0.5])}%",
            0.5: f"Q3 RH={round(humids[0.5])}%-{round(humids[0.75])}%",
            0.75: f"Q4 RH={round(humids[0.75])}%-{round(humids[1])}%",
        },
        "color": {
            0.0: "#1f77b4",  # muted blue
            0.25: "#ff7f0e",  # orange
            0.5: "#d62728",  # red
            0.75: "#6a3d9a",  # purple
        },
    },
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureF4.png")

In [ ]:
CRs_ginis_RH = pd.DataFrame(index=PROJECTS_2022_TOTAL, columns=[0, 0.25, 0.5, 0.75])
for project in list(CRs_ginis_RH.index):
    for bucket in list(CRs_ginis_RH.columns):
        CRs_ginis_RH.loc[project, bucket] = gini(CRs_bucketed_df_RH[project][bucket])

# Heating requests

In [ ]:
HRs = pull_from_dataset(
    "2023",
    PROJECTS_2022,
    "zone-simple_heating_requests",
)

In [ ]:
HRs = add_total_to_dfs(HRs)

## OAT

In [ ]:
HRs_bucketed_temp = {}
HRs_lorenz_bucketed_temp = {}
for project in HRs:
    HRs_bucketed_temp[project] = {}
    HRs_lorenz_bucketed_temp[project] = {}
    for bucket in hours_temp:
        these_HRs = HRs[project].loc[hours_temp[bucket], :].sum()
        HRs_bucketed_temp[project][bucket] = these_HRs.to_frame(name=bucket)
        these_HRs = these_HRs.sort_values(ascending=False)
        lorenz = these_HRs.cumsum() / these_HRs.sum()
        HRs_lorenz_bucketed_temp[project][bucket] = lorenz.to_frame(name=bucket)

In [ ]:
HRs_bucketed_df_temp = {}
HRs_lorenz_bucketed_df_temp = {}
for project in HRs_bucketed_temp:
    these_HRs = HRs_bucketed_temp[project]
    df = pd.concat([these_HRs[bucket] for bucket in these_HRs], axis=1)
    HRs_bucketed_df_temp[project] = df

    these_lorenz = HRs_lorenz_bucketed_temp[project]
    df = pd.concat([these_lorenz[bucket] for bucket in these_lorenz], axis=1)
    HRs_lorenz_bucketed_df_temp[project] = df

In [ ]:
fig = viz.make_dot_plot(
    y_data=HRs_lorenz_bucketed_df_temp,
    sort_by="all",
    normalize_x=True,
    y_axis_title="Fraction of Cumulative<br>Heating Requests [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.075,
    horizontal_spacing=0.1,
    color_legend={
        "name": {
            0.0: f"Q1 OAT={round(F_to_C(temps[0]))}°C-{round(F_to_C(temps[0.25]))}°C",
            0.25: f"Q2 OAT={round(F_to_C(temps[0.25]))}°C-{round(F_to_C(temps[0.5]))}°C",
            0.5: f"Q3 OAT={round(F_to_C(temps[0.5]))}°C-{round(F_to_C(temps[0.75]))}°C",
            0.75: f"Q4 OAT={round(F_to_C(temps[0.75]))}°C-{round(F_to_C(temps[1]))}°C",
        },
        "color": {
            0.0: "#1f77b4",  # muted blue
            0.25: "#ff7f0e",  # orange
            0.5: "#d62728",  # red
            0.75: "#6a3d9a",  # purple
        },
    },
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureF3.png")

In [ ]:
HRs_ginis_temp = pd.DataFrame(index=PROJECTS_2022_TOTAL, columns=[0, 0.25, 0.5, 0.75])
for project in list(HRs_ginis_temp.index):
    for bucket in list(HRs_ginis_temp.columns):
        HRs_ginis_temp.loc[project, bucket] = gini(
            HRs_bucketed_df_temp[project][bucket]
        )

## RH

In [ ]:
HRs_bucketed_RH = {}
HRs_lorenz_bucketed_RH = {}
for project in HRs:
    HRs_bucketed_RH[project] = {}
    HRs_lorenz_bucketed_RH[project] = {}
    for bucket in hours_temp:
        these_HRs = HRs[project].loc[hours_RH[bucket], :].sum()
        HRs_bucketed_RH[project][bucket] = these_HRs.to_frame(name=bucket)
        these_HRs = these_HRs.sort_values(ascending=False)
        lorenz = these_HRs.cumsum() / these_HRs.sum()
        HRs_lorenz_bucketed_RH[project][bucket] = lorenz.to_frame(name=bucket)

In [ ]:
HRs_bucketed_df_RH = {}
HRs_lorenz_bucketed_df_RH = {}
for project in HRs_bucketed_RH:
    these_HRs = HRs_bucketed_RH[project]
    df = pd.concat([these_HRs[bucket] for bucket in these_HRs], axis=1)
    HRs_bucketed_df_RH[project] = df

    these_lorenz = HRs_lorenz_bucketed_RH[project]
    df = pd.concat([these_lorenz[bucket] for bucket in these_lorenz], axis=1)
    HRs_lorenz_bucketed_df_RH[project] = df

In [ ]:
fig = viz.make_dot_plot(
    y_data=HRs_lorenz_bucketed_df_RH,
    sort_by="all",
    normalize_x=True,
    y_axis_title="Fraction of Cumulative<br>Heating Requests [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.075,
    horizontal_spacing=0.1,
    color_legend={
        "name": {
            0.0: f"Q1 RH={round(humids[0])}%-{round(humids[0.25])}%",
            0.25: f"Q2 RH={round(humids[0.25])}%-{round(humids[0.5])}%",
            0.5: f"Q3 RH={round(humids[0.5])}%-{round(humids[0.75])}%",
            0.75: f"Q4 RH={round(humids[0.75])}%-{round(humids[1])}%",
        },
        "color": {
            0.0: "#1f77b4",  # muted blue
            0.25: "#ff7f0e",  # orange
            0.5: "#d62728",  # red
            0.75: "#6a3d9a",  # purple
        },
    },
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureF5.png")

In [ ]:
HRs_ginis_RH = pd.DataFrame(index=PROJECTS_2022_TOTAL, columns=[0, 0.25, 0.5, 0.75])
for project in list(HRs_ginis_RH.index):
    for bucket in list(HRs_ginis_RH.columns):
        HRs_ginis_RH.loc[project, bucket] = gini(HRs_bucketed_df_RH[project][bucket])

# Gini coefficient summaries

## OAT

In [ ]:
fig = viz.make_bar_plot(
    y_data={
        "Cooling Requests": CRs_ginis_temp,
        "Heating Requests": HRs_ginis_temp,
    },
    bar_mode="group",
    y_axis_title="Gini Coefficient [Unitless]",
    bar_legend={
        "name": {
            0.0: f"Q1 OAT={round(F_to_C(temps[0]))}°C-{round(F_to_C(temps[0.25]))}°C",
            0.25: f"Q2 OAT={round(F_to_C(temps[0.25]))}°C-{round(F_to_C(temps[0.5]))}°C",
            0.5: f"Q3 OAT={round(F_to_C(temps[0.5]))}°C-{round(F_to_C(temps[0.75]))}°C",
            0.75: f"Q4 OAT={round(F_to_C(temps[0.75]))}°C-{round(F_to_C(temps[1]))}°C",
        },
        "color": {
            0.0: "#1f77b4",  # muted blue
            0.25: "#ff7f0e",  # orange
            0.5: "#d62728",  # red
            0.75: "#6a3d9a",  # purple
        },
    },
    width=850,
    height=550,
    y_range=[0, 1],
    legend_order="forward",
    title_size=SINGLE_PLOT_LEGEND_SIZE + 2,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.25,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure12.png")

## RH

In [ ]:
fig = viz.make_bar_plot(
    y_data={
        "Cooling Requests": CRs_ginis_RH,
        "Heating Requests": HRs_ginis_RH,
    },
    bar_mode="group",
    y_axis_title="Gini Coefficient [Unitless]",
    bar_legend={
        "name": {
            0.0: f"Q1 RH={round(humids[0])}%-{round(humids[0.25])}%",
            0.25: f"Q2 RH={round(humids[0.25])}%-{round(humids[0.5])}%",
            0.5: f"Q3 RH={round(humids[0.5])}%-{round(humids[0.75])}%",
            0.75: f"Q4 RH={round(humids[0.75])}%-{round(humids[1])}%",
        },
        "color": {
            0.0: "#1f77b4",  # muted blue
            0.25: "#ff7f0e",  # orange
            0.5: "#d62728",  # red
            0.75: "#6a3d9a",  # purple
        },
    },
    width=850,
    height=550,
    y_range=[0, 1],
    legend_order="forward",
    title_size=SINGLE_PLOT_LEGEND_SIZE + 2,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)

fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.25,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureF1.png")